## Part 3: Bragg Peak Masking

Because different corrections are needed when integrating Bragg peaks and diffuse data, it is important to ensure that the signals are separated before processing. It is important to note that a Bragg peaks' extent in $\phi$ can vary depending on the distance from the spindle axis in reciprocal space. And, Bragg peaks may not be perfectly symmetric Gaussian peaks. Peak width depends on crystal mosaicity, and this can even vary during a scan. In addition, the diffraction geometry may not be modeled exactly right, leading to further errors in peak position. A masking function must account for all of these effects. 

Designing a the perfect Bragg peak mask is a tricky problem, and different researchers have approached it various ways. With _mdx2_ we take a simple empirical approach. Here is the algorithm:

- List all of the pixels in the image stack above a certain count threshold ("hot pixels").
- Index all of the hot pixels, converting their position in image space to Miller index (_h_, _k_, _l_).
- Subtract the nearest integer from each Miller index leaving only the fractional part (_Δh_, _Δk_, _Δl_) which varies from -0.5 to 0.5. Because peaks have finite extent, the values of _Δh_, _Δk_, _Δl_ will not all be zero, but will look like a cloud of points.
- Fit an anisotropic 3D Gaussian distribution to the cloud of points.
- Flag outliers beyond a certain sigma threshold ("bad pixels")
- Create a mask for all pixels in the image stack excluding outliers and pixels falling within the 3D Gaussian region according to a sigma threshold.

To execute the masking altorithm in _mdx2_, first perform the strong pixel search:

In [1]:
!mdx2.find_peaks geometry.nxs data.nxs --count_threshold 20

Reading miller_index from geometry.nxs
  importing as MillerIndex from mdx2.geometry
Reading image_series from data.nxs
  importing as ImageSeries from mdx2.data
finding pixels with counts above threshold: 20.0
found 8 peaks in chunk 1
found 22 peaks in chunk 2
found 31 peaks in chunk 3
found 67 peaks in chunk 6
found 246 peaks in chunk 7
found 80 peaks in chunk 8
found 5 peaks in chunk 9
found 4 peaks in chunk 10
found 272 peaks in chunk 11
found 683 peaks in chunk 12
found 340 peaks in chunk 13
found 19 peaks in chunk 14
found 48 peaks in chunk 15
found 474 peaks in chunk 16
found 938 peaks in chunk 17
found 638 peaks in chunk 18
found 75 peaks in chunk 19
found 77 peaks in chunk 20
found 599 peaks in chunk 21
found 967 peaks in chunk 22
found 783 peaks in chunk 23
found 123 peaks in chunk 24
found 124 peaks in chunk 25
found 434 peaks in chunk 26
found 411 peaks in chunk 27
found 632 peaks in chunk 28
found 123 peaks in chunk 29
found 90 peaks in chunk 30
found 440 peaks in chunk 31

A threshold of 20 was chosen to be 10 times the background rate of ~ 2 photons / pixel, which works well in this case. When the search finishes, a Gaussian is automatically fit to the point cloud and the results are saved to a file (`peaks.nxs`) and printed.

Note that the peak center (`r0`) is not exactly zero, which may indicate a slight error in the diffraction geometry. This error, as well as the extent of the point cloud (`sigma`), should be taken into account when designing a reciprocal space grid for integration. The sampling should not be finer than the geometric errors allow. Here, the Bragg peaks are well localized, and sampling as fine as 20 x 20 x 20 voxels per Bragg peak could be justified (0.5/0.025 = 20). Note that such fine maps consume a lot of computer memory, and thus for the purposes of this tutorial, a coarser map will be used.

Next, create the mask:

In [2]:
!mdx2.mask_peaks geometry.nxs data.nxs peaks.nxs --sigma_cutoff 3

Reading miller_index from geometry.nxs
  importing as MillerIndex from mdx2.geometry
Reading image_series from data.nxs
  importing as ImageSeries from mdx2.data
Reading peak_model from peaks.nxs
  importing as GaussianPeak from mdx2.geometry
Reading peaks from peaks.nxs
  importing as Peaks from mdx2.data
masking peaks with sigma above threshold: 3.0
indexing chunk 0
indexing chunk 1
indexing chunk 2
indexing chunk 3
indexing chunk 4
indexing chunk 5
indexing chunk 6
indexing chunk 7
indexing chunk 8
indexing chunk 9
indexing chunk 10
indexing chunk 11
indexing chunk 12
indexing chunk 13
indexing chunk 14
indexing chunk 15
indexing chunk 16
indexing chunk 17
indexing chunk 18
indexing chunk 19
indexing chunk 20
indexing chunk 21
indexing chunk 22
indexing chunk 23
indexing chunk 24
indexing chunk 25
indexing chunk 26
indexing chunk 27
indexing chunk 28
indexing chunk 29
indexing chunk 30
indexing chunk 31
indexing chunk 32
indexing chunk 33
indexing chunk 34
indexing chunk 35
indexing

The `sigma_cutoff` parameter controlls the size of the masked region around the Bragg peak. A value of 3 means that pixels are masked within 3 standard deviations of the peak center according to the model fit above. The function also masks outliers flagged previously. The resulting masks are saved as an image stack in `mask.nxs` and should be inspected using _NeXpy_ before proceeding.

Note that in _mdx2_ version 0.3.1, `mdx2.mask_peaks` does not account for systematic absences. This is a particular issue for space group I 2<sub>1</sub>3, because the reflection condition  $h+k+l=2n$ means that half of the Bragg peaks are absent. Although the mask generated above excludes twice as many spots as necessary, in this case relatively few pixels are excluded so the effect on the final map is minimal.
